In [1]:
import torch
from sklearn.metrics import f1_score, accuracy_score
from training_structures.unimodal import test as uni_test
from training_structures.Supervised_Learning import test
from commons import get_dataloader, get_ckp_name

In [2]:
def test_ckp(dataset, name, suffix='best', unimodal=True, balanced=False):
    print(f'{name.upper()} on {dataset.upper()}:')

    traindata, valdata, testdata = get_dataloader(dataset, task='classification',
                                                  num_workers=1, max_pad=unimodal, balanced=balanced)

    if unimodal:
        encoder = torch.load(get_ckp_name(dataset, name, suffix=suffix, encoder=True)).cuda()
        head = torch.load(get_ckp_name(dataset, name, suffix=suffix, head=True)).cuda()
        results = uni_test(encoder, head, testdata, dataset, method_name=name,
                           task='multilabel', modalnum=1, no_robust=True)
    else:
        model = torch.load(get_ckp_name(dataset, name, suffix=suffix)).cuda()
        results = test(model=model, test_dataloaders_all=testdata, dataset=dataset,
                       is_packed=True,  criterion=torch.nn.BCEWithLogitsLoss(),
                       task='multilabel', no_robust=True)

    true = results['true']
    pred = results['pred']

    f1 = f1_score(true, pred)
    acc = accuracy_score(true, pred)
    results['F1'] = f1
    results['Accuracy'] = acc

    print('F1 binary:', f1)
    print(' Accuracy:', acc)

    return results

In [3]:
result_lf_c       = test_ckp('mosei', 'lf_c', unimodal=False)
result_tr_c_mosei = test_ckp('mosei', 'tr_c')
result_tr_c_emo2  = test_ckp('emo2', 'tr_c')

LF_C on MOSEI:
 f1_micro: 0.7237237237237237 f1_macro: 0.723452497977242
Inference Time: 22.660237073898315
Inference Params: 50443543
F1 binary: 0.7321131447587356
 Accuracy: 0.7237237237237237
TR_C on MOSEI:
 f1_micro: 0.5832260832260833 f1_macro: 0.5807399301738184
Inference Time: 7.6084136962890625
Inference Params: 4955897
F1 binary: 0.5484545665814547
 Accuracy: 0.5832260832260833
TR_C on EMO2:
Train size: 391
  Val size: 97
 Test size: 2171
 Total pos: 122
 f1_micro: 0.987563334868724 f1_macro: 0.9372479539238094
Inference Time: 4.318965435028076
Inference Params: 4955897
F1 binary: 0.8810572687224669
 Accuracy: 0.987563334868724


In [3]:
result_tr_c_emo2_balanced = test_ckp('emo2', 'tr_c', balanced=True)

TR_C on EMO2:
Train size: 391
  Val size: 97
 Test size: 122
 Total pos: 122
 f1_micro: 0.8852459016393442 f1_macro: 0.4695652173913043
Inference Time: 4.949899673461914
Inference Params: 4955897
F1 binary: 0.9391304347826086
 Accuracy: 0.8852459016393442
